In [1]:
import math
import numpy as np
import cmath 



In [2]:
fo = 10.7e6
wo = 2*np.pi*fo
# Qo de los transformadores
Qo = 100

# MMBT918 Ic = 8mA, VCE=10V, fo=10.7MHz
y11 =   1.80e-3  + 0.85e-3*1j
y12 = - 0.01e-3  - 0.08e-3*1j
y21 =    110e-3  -   20e-3*1j 
y22 =   0.25e-3  + 0.12e-3*1j

# MMBT918 Ic = 4mA, VCE=10V, fo=10.7MHz
y11 =   1.30e-3  + 0.80e-3*1j
y12 = - 0.01e-3  - 0.08e-3*1j
y21 =     95e-3  -   19e-3*1j 
y22 =   0.08e-3  + 0.20e-3*1j

ytr = np.matrix([[y11, y12],[ y21, y22]])


In [3]:
def printY(y):
    print( 'y11= {0.real:4.2E}+j({0.imag:4.2E})' .format(y[0,0]) )
    print( 'y12= {0.real:4.2E}+j({0.imag:4.2E})' .format(y[0,1]) )
    print( 'y21= {0.real:4.2E}+j({0.imag:4.2E})' .format(y[1,0]) )
    print( 'y22= {0.real:4.2E}+j({0.imag:4.2E})' .format(y[1,1]) )

def printYp(y):
    print('Polar')
    print( '|y11|= {0:6.4E}, <y11= {1:6.1F}' .format(abs(y[0,0]), cmath.phase(ytr[0,0])*180./np.pi) )
    print( '|y12|= {0:6.4E}, <y12= {1:6.1F}' .format(abs(y[0,1]), cmath.phase(ytr[0,1])*180./np.pi) )
    print( '|y21|= {0:6.4E}, <y21= {1:6.1F}' .format(abs(y[1,0]), cmath.phase(ytr[1,0])*180./np.pi) )
    print( '|y22|= {0:6.4E}, <y22= {1:6.1F}' .format(abs(y[1,1]), cmath.phase(ytr[1,1])*180./np.pi) )

    
def getSpice(y):
    print( '.param g11= {0.real:6.4}' .format(y[0,0]) )
    print( '.param g12= {0.real:6.4E}' .format(y[0,1]) )
    print( '.param g21= {0.real:6.4E}' .format(y[1,0]) )
    print( '.param g22= {0.real:6.4E}' .format(y[1,1]) )
    print( '.param c11= {0:6.4E}' .format(y[0,0].imag/(wo)) )
    print( '.param c12= {0:6.4E}' .format((y[0,1].imag)/(wo)) )
    print( '.param c21= {0:6.4E}' .format((y[1,0].imag)/(wo)) )
    print( '.param c22= {0:6.4E}' .format(y[1,1].imag/(wo)) )
    
def printInOut(yg,yl):
    print('######## Entrada #########')
    print( 'rin= {0:4.2E}' .format(1/yg.real) )
    print( 'cin= {0:4.2E}' .format(-yg.imag/wo) )
    print('######## Salida #########')
    print( 'ro= {0:4.2E}' .format(1/yl.real) )
    print( 'co= {0:4.2E}' .format(-yl.imag/wo) )


In [3]:
def linvill(y):
    ya = y[0,1]*y[1,0]
    C = ( abs(ya) + ya.real )/ (2*y[0,0].real*y[1,1].real)
    print('###############  LINVILL   #####################')
    print( 'coeficiente de linvill C= {:4.2F}' .format(C))
    if C <= 1: print( 'si C es menor a uno es incondicionalmente estable')
    else    : print( 'si C es mayor o igula a 1 es potencialmente inestable')
    
    
def stern(y, yg, yl):
    ya = y[0,1]* y[1,0]
    K = (2*(yg.real+y[0,0].real)*(yl.real+y[1,1].real))/( abs(ya) + ya.real )
    print('###############  Stern   #####################')
    print('coeficiente de Stern K=', K)
    print('si k es MAYOR a 1 no oscilara' )   

def Ar(y,yg,yo): # vo/ig
    ya = y[0,1]* y[1,0]
    Ar = -y[1,0]/((y[0,0]+yg)*(y[1,1]+yl)-ya)
    print('###############  Ganancia de transresistencia (vo/ig)   #########')
    print( 'vo/ig = {:4.2F}' .format(abs(Ar)))
    print( 'vo/ig = {:4.2F} dB' .format(20*np.log10(abs(Ar)))) 
    
def At(y,yg,yl) :  
    ya = y[0,1]* y[1,0]
    pgain = 4*yg.real*yl.real*abs(y[1,0])**2 / (abs((yl+y[1,1])*(y[0,0]+yg)-ya))**2
    print('###############  Ganancia de potencia   #####################')
    print( 'Power Gain = {:4.2F}' .format(pgain))
    print( 'Power Gain = {:4.2F} dB' .format(10*np.log10(pgain)))    
    
    
def neutraliza(y,wo,n = 1):
    Cn = -y[0,1].imag/wo*n
    y11 =  wo*Cn*1j
    y12 =  wo*Cn*1j/n
    y21 =  wo*Cn*1j/n
    y22 =  wo*Cn*1j/n**2
    yn  = np.matrix([[y11, y12],[ y21, y22]])
    # cuadripolo resultante
    yt  = y + yn
    print('cap=',Cn, 'n=', n)
    return (yt,Cn)

def yg_yl(y):
    yl = y[1,1].real - y[1,1].imag*1j
    for i in range(10):
        yin  = y[0,0] - y[0,1]*y[1,0]/(y[1,1]+yl)
        yg   = yin.real - yin.imag*1j
        yout = y[1,1] - y[0,1]*y[0,1]/(y[0,0]+yg)
        yl   = yout.real - yout.imag*1j
    return (yg, yl)

def yg_ylQ(y,Qc,Qo,wo):
    yl = y[1,1].real - y[1,1].imag*1j
    for i in range(10):
        yin  = y[0,0] - y[0,1]*y[1,0]/(y[1,1]+yl)
        yg   = yin.real - yin.imag*1j
        yout = y[1,1] - y[0,1]*y[0,1]/(y[0,0]+yg)
        yl   = yout.real - yout.imag*1j
    return (yg, yl)

    




def divC(Q,wo,Rin,RL):
    Qm = ((RL/Rin)*(1+Q**2)-1)**0.5
    C  = Q/Rin/wo
    Cs = C*(1+1/Q**2)
    Cb = Qm/R/wo
    Cbs = Cb*(1+1/Qm**2)
    Ca = 1/(1/Cs-1/Cbs)
    return(Ca,Cb)


def yf(y,Qc,Qo,wo,w):
    rext = 1/y.real/2
    bl = (1/Qc-1/Qo)*rext
    return (1/(Qc*bl))*(1+1j*Qc*(w/wo-wo/w))

def yin(y,ys):
    return y[0,0] - (y[0,1]*y[1,0])/(ys)

def yout(y,ys):
    return y[1,1] - (y[0,1]*y[1,0])/(ys)

def yw(y,Qo,Q1,w1,Q2,w2, n=10):
    yo = y[1,1].real - y[1,1].imag*1j
    for i in range(n):
        ys  = yf(yo,Q2,Qo,w2,w1)
        yi  = yin(y,ys)
        ys  = yf(yi,Q1,Qo,w1,w2)
        yo  = yout(y,ys)
    return (yi, yo)
            
def getCom3(yo, rl,wo, Qc, Qo, nf):
    Qs   = 1/(1/Qc-1/Qo)
    yot   = yo/nf**2 
    rext  = (1/yot.real)/2
    L     = (rext/Qs)/wo
    C     = 1/(wo**2*L) 
    Lf    = L / nf**2 
    Lp    = L / (nf/(nf-1))**2
    ns    = ((2*rext)/rl)**0.5
    Ls    = L / ns**2 
    C     = C - (yot.imag)/wo 
    return (Lp,Lf,Ls,C,ns,rext)




def getCom2(yo, yi, wo, Qc, Qo, nf):
    Qs    = 1/(1/Qc-1/Qo)
    yot   = yo/nf**2 
    rext  = (1/yot.real)/2
    L     = (rext/Qs)/wo
    C     = 1/(wo**2*L) 
    Lf    = L / nf**2 
    Lp    = L / (nf/(nf-1))**2
    r     = 1/yi.real
    ns    = ((2*rext)/r)**0.5
    Ls    = L / ns**2 
    C     = C - (yot.imag)/wo - (yi.imag/ns**2)/wo  
    return (Lp,Lf,Ls,C,ns,rext)


def getCom1(rg, yi,wo, Qc, Qo, L):
    Qs    = 1/(1/Qc-1/Qo)
    C     = 1/(wo**2*L) 
    rext  = L*wo*Qs 
    np    = ((2*rext)/rg)**0.5
    Lp1  = L /(np/(np-1))**2
    Lp2  = L / np**2
    ns    = ((2*rext)/(1/yi.real))**0.5
    Ls    = L / ns**2
    C     = C   - (yi.imag/ns**2)/wo
    return (Lp,Lf,Ls,C,n,rext)

In [5]:
printY(ytr)

# calculo linvill del transistor
linvill(ytr)


# admitancia de entrada y de salida para MTE
(yg,yl) = yg_yl(ytr) 
printInOut(yg,yl)

y11= 1.30E-03+j(8.00E-04)
y12= -1.00E-05+j(-8.00E-05)
y21= 9.50E-02+j(-1.90E-02)
y22= 8.00E-05+j(2.00E-04)
###############  LINVILL   #####################
coeficiente de linvill C= 25.68
si C es mayor o igula a 1 es potencialmente inestable
######## Entrada #########
rin= 5.99E+01
cin= 7.01E-10
######## Salida #########
ro= 1.25E+04
co= 2.98E-12


In [6]:
# neutralizador
nf = 6
(yu,Cp) = neutraliza(ytr,wo,nf)

printY(yu)
print(1/yu[0,0].real)
print(1/yu[1,1].real)
print(yu[0,1]*yu[1,0])
# calculo linvill del transistor
linvill(yu)

# admitancia de entrada y de salida para MTE
(yg,yl) = yg_yl(yu) 
printInOut(yg,yl)

# stern 
stern(yu, yg, yl)


Ar(yu,yg,yl)
At(yu,yg,yl)

cap= 7.13966099851e-12 n= 6
y11= 1.30E-03+j(1.28E-03)
y12= -1.00E-05+j(0.00E+00)
y21= 9.50E-02+j(-1.89E-02)
y22= 8.00E-05+j(2.13E-04)
769.230769231
12500.0
(-9.5e-07+1.892e-07j)
###############  LINVILL   #####################
coeficiente de linvill C= 0.09
si C es menor a uno es incondicionalmente estable
######## Entrada #########
rin= 1.38E+02
cin= 1.45E-12
######## Salida #########
ro= 1.25E+04
co= 3.17E-12
###############  Stern   #####################
coeficiente de Stern K= 146.428934458
si k es MAYOR a 1 no oscilara
###############  Ganancia de transresistencia (vo/ig)   #########
vo/ig = 41825.05
vo/ig = 92.43 dB
###############  Ganancia de potencia   #####################
Power Gain = 4051.12
Power Gain = 36.08 dB


Tengo 3 etapas sintonizadas, realizadas con 2 transistores.

1 SS : Qca = 53.5 foa = 10.52679 MHz

2 SS : Qcc = 53.5 fob = 10.87320 MHz

3 SS : Qcb = 26.7 foc = 10.70000 MHz

In [18]:
rg = 1500
rl = 1500

Qo =100

#[(53.500000000000007, 10526794.919243112), (26.75, 10700000.0), (53.500000000000007, 10873205.080756888)]
Qcc = 53.5
Qcb = 53.5
Qca = 26.75
woc = 10526794.91*2*math.pi
wob = 10873205.08*2*math.pi
woa = wo

(yi1,yo1) = yw(yu,Qo,Qca,woa,Qcb,wob,20)

print('yi1=',1/yi1.real)
print('yo1=',1/yo1.real)
# stern 
stern(yu, yi1, yo1)

(yi2,yo2) = yw(yu,Qo,Qcb,wob,Qcc,woc,20)

print('yi2=',1/yi2.real)
print('yo2=',1/yo2.real)
# stern 
stern(yu, yi2, yo2)
# dado el transistor
ri = 1/yg.real
bi = -yg.imag
ro = 1/yl.real
bo = -yl.imag
print(ri,ro)

yi1= 579.432866799
yo1= 5677.11350128
###############  Stern   #####################
coeficiente de Stern K= 83.0837915102
si k es MAYOR a 1 no oscilara
yi2= 739.649608477
yo2= 9878.60231226
###############  Stern   #####################
coeficiente de Stern K= 51.5210468363
si k es MAYOR a 1 no oscilara
138.160891227 12501.7958798


In [26]:
# calculo de los inductores
# segundo
Qsb   = 1/(1/Qcb-1/Qo)
ro    = 1/yo1.real 
rextb = (ro*nf**2)/2


Lb    = (1/Qsb)*rextb/wob 
Csb   = 1/(wob**2*Lb) 

Lbf   = Lb / nf**2 
Lbp   = Lb / (nf/(nf-1))**2

n2    = ((2*rextb)/(1/yi2.real))**0.5


Lbs   = Lb / n2**2 

Cb    = Csb - (yo1.imag/nf**2)/wob - (yi2.imag/n2**2)/wob

# tercero
Qsc   = 1/(1/Qcc-1/Qo)
ro    = 1/yo2.real 
rextc = (ro*nf**2)/2

Lc    = (rextc/Qsc)/woc
Csc   = 1/(woc**2*Lc) 

Lcf   = Lc / nf**2 
Lcp   = Lc / (nf/(nf-1))**2

n3    = ((2*rextc)/(1500))**0.5
Lcs   = Lc / n3**2 

Cc    = Csc - (yo2.imag/nf**2)/woc 

print(Lc,Lcp,Lcf,Lcs)

# primero
La     = Lc
Csa    = 1/(woa**2*La) 
Qsa    = 1/(1/Qca-1/Qo)
rexta  = La*woa*Qsa 

ni    = ((2*rexta)/(1500))**0.5

Lap1  = La /(ni/(ni-1))**2
Lap2  = La / ni**2

n1    = ((2*rexta)/(1/yi1.real))**0.5
Las   = La / n1**2
Ca    = Csa  - (yi1.imag/n1**2)/woa




2.33663645466e-05 1.62266420463e-05 6.49065681851e-07 9.85563030074e-08


In [27]:
getSpice(yu)

.param g11= 0.0013
.param g12= -1.0000E-05
.param g21= 9.5000E-02
.param g22= 8.0000E-05
.param c11= 1.9039E-11
.param c12= 0.0000E+00
.param c21= -2.8142E-10
.param c22= 3.1732E-12


In [28]:
# sintonizado entrada
print( '.param Lap1= {0:6.4E}' .format(Lap1))
print( '.param Lap2= {0:6.4E}' .format(Lap2))
print( '.param Las = {0:6.4E}' .format(Las))
print( '.param Ca  = {0:6.4E}' .format(Ca))
print( '.param rpa= {0:6.4E}' .format(Qo*Las*woa))


.param Lap1= 1.8328E-05
.param Lap2= 3.0548E-07
.param Las = 1.1800E-07
.param Ca  = 9.3362E-12
.param rpa= 7.9334E+02


In [29]:
# sintonizado intermedio
print( '.param Lbp  = {0:6.4E}' .format(Lbp))
print( '.param Lbf = {0:6.4E}' .format(Lbf))
print( '.param Lbs = {0:6.4E}' .format(Lbs))
print( '.param Cb  = {0:6.4E}' .format(Cb))
print( '.param Cp  = {0:6.4E}' .format(Cp))
print( '.param rpb = {0:6.4E}' .format(Qo*Lbs*wob))


.param Lbp  = 9.0282E-06
.param Lbf = 3.6113E-07
.param Lbs = 4.7050E-08
.param Cb  = 1.6408E-11
.param Cp  = 7.1397E-12
.param rpb = 3.2144E+02


In [30]:
# sintonizado tercer
print( '.param Lcp  = {0:6.4E}' .format(Lcp))
print( '.param Lcf = {0:6.4E}' .format(Lcf))
print( '.param Lcs = {0:6.4E}' .format(Lcs))
print( '.param Cc  = {0:6.4E}' .format(Cc))
print( '.param rpc = {0:6.4E}' .format(Qo*Lcs*woc))

.param Lcp  = 1.6227E-05
.param Lcf = 6.4907E-07
.param Lcs = 9.8556E-08
.param Cc  = 9.6758E-12
.param rpc = 6.5187E+02
